# Project Description

Write a solution to find the prices of all products on 2019-08-16. Assume the price of all products before any change is 10.

- **Return the result table in any order.**

## Example

### Input:
**Products table:**
| product_id | new_price | change_date |
|------------|-----------|-------------|
| 1          | 20        | 2019-08-14  |
| 2          | 50        | 2019-08-14  |
| 1          | 30        | 2019-08-15  |
| 1          | 35        | 2019-08-16  |
| 2          | 65        | 2019-08-17  |
| 3          | 20        | 2019-08-18  |

### Output:
| product_id | price |
|------------|-------|
| 2          | 50    |
| 1          | 35    |
| 3          | 10    |

# Intuition
The problem requires us to find the price of each product on a specific date. We need to consider that prices might change over time, and for products without any price change before or on that date, we should assume the price is 10. Our intuition would be to find the most recent price update for each product before or on the target date.

# Approach
1. **Use a CTE (Common Table Expression)** to rank the price changes for each product based on the date, in descending order. This gives us the latest price change for each product before or on the specified date.
2. **Select from this CTE** where the rank is 1 (the latest price), to get the current price for products that had changes.
3. **Use UNION** to combine this with products that didn't have any price change before or on the date, assigning them the default price of 10.

I saw other solutions on Leetcode from other leetcocers but these were less efficient involving joins and correlated subqueries. For instance:

```sql
SELECT 
    DISTINCT p1.product_id,
    IFNULL(p2.new_price, 10) AS price
FROM 
    Products p1
LEFT JOIN 
    Products p2 ON p1.product_id = p2.product_id 
    AND p2.change_date = (SELECT MAX(change_date) 
                          FROM Products p3 
                          WHERE p3.product_id = p1.product_id 
                          AND p3.change_date <= '2019-08-16')



SELECT p.product_id, 
       COALESCE((
           SELECT new_price 
           FROM Products 
           WHERE product_id = p.product_id AND change_date <= '2019-08-16'
           ORDER BY change_date DESC 
           LIMIT 1
       ), 10) AS price
FROM (SELECT DISTINCT product_id FROM Products) p;

```
# Winner
```sql

WITH cte AS (
    SELECT *,
           RANK() OVER (PARTITION BY product_id ORDER BY change_date DESC) AS r
    FROM Products
    WHERE change_date <= '2019-08-16'
)
SELECT product_id, new_price AS price
FROM cte
WHERE r = 1
UNION
SELECT product_id, 10 AS price
FROM Products
WHERE product_id NOT IN (SELECT product_id FROM cte)
```

# Intuition

##### First, grab the relevant data: 
We use a CTE to filter all price changes that occurred on or before the specified date ('2019-08-16'). This reduces the dataset we need to work with, potentially improving performance.
Rank the changes: 
Within this subset, we rank each change by date. Using RANK() OVER (PARTITION BY product_id ORDER BY change_date DESC), we ensure that for each product, the most recent change gets rank 1, the next most recent gets rank 2, and so on.

##### Select the latest price:
By selecting from this CTE where r = 1, we only get the latest price change for each product that had changes before or on the date.
Handle products without price changes:
The UNION operation then adds products which did not have any price change recorded by the date. These are identified by checking if their product_id is not in the CTE (meaning they had no price change). We assign these a default price of 10.

# Complexity
### Time complexity:
O(nlogn) where n is the number of rows in the Products table. This is due to the sorting involved in the window function for ranking.
### Space complexity:
O(n) for storing the CTE results before final selection.